# 字符串序列的处理
1. 字符串拆分
2. 诊断是否包含 子字符串
3. 替换
4. 类别字符串转换为离散变量
5. 字符串去重查看

In [1]:
# coding = utf-8
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import time # 用于计算程序 处理耗时

In [2]:
data = pd.read_csv('911.csv')
data.head(2)

,lat,lng,desc,zip,title,timeStamp,twp,addr,e
0,40.297876,-75.581294,REINDEER CT & DEAD END; NEW HANOVER; Station ...,19525.0,EMS: BACK PAINS/INJURY,2015-12-10 17:10:52,NEW HANOVER,REINDEER CT & DEAD END,1
1,40.258061,-75.264680,BRIAR PATH & WHITEMARSH LN; HATFIELD TOWNSHIP...,19446.0,EMS: DIABETIC EMERGENCY,2015-12-10 17:29:21,HATFIELD TOWNSHIP,BRIAR PATH & WHITEMARSH LN,1


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 423909 entries, 0 to 423908
Data columns (total 9 columns):
lat          423909 non-null float64
lng          423909 non-null float64
desc         423909 non-null object
zip          371780 non-null float64
title        423909 non-null object
timeStamp    423909 non-null object
twp          423750 non-null object
addr         423909 non-null object
e            423909 non-null int64
dtypes: float64(3), int64(1), object(5)
memory usage: 21.0+ MB


# 1. **字符串拆分**

    1. 先诊断每行是否都是统一的形式
    2. 再拆分
    
    

* 注意Series 的tolist() 没有下划线
* DataFrame 的to_list() 有下划线
* data.assign(new_col=) 增加一列
    * 因为 有时直接复制会报错

In [4]:
(data['title'].str.split(': ').map(len) != 2).value_counts()

False    423909
Name: title, dtype: int64

In [5]:
# 方法一,拆分为DataFrame选取指定列
data = data.assign(type = data['title'].str.split(": ",expand=True)[0])

print(data['type'].value_counts())

EMS        208676
Traffic    151458
Fire        63775
Name: type, dtype: int64


In [6]:
# 方法二
a = data['title'].str.split(": ")
print(a.head(2))
list(set([i[0] for i in a]))

0     [EMS, BACK PAINS/INJURY]
1    [EMS, DIABETIC EMERGENCY]
Name: title, dtype: object


['Fire', 'EMS', 'Traffic']

In [7]:
# 方法二 tolist()
b = data['title'].str.split(": ").tolist()
list(set([i[0] for i in b]))

['Fire', 'EMS', 'Traffic']

# 2. **诊断是否包含子字符串**

In [8]:
data['title'].str.contains('EMS').sum() # 紧急电话中包含EMS的事件类型次数

208682

In [9]:
(~data['title'].str.contains('EMS')).sum() # 不包含EMS的时间类型次数

215227

# 3. **替换**


* 注意用时
    * 可是在 讲解中 说的是方法一会比较快——应该是这里的数据太少的原因

In [10]:
# 方法一
start = time.clock()

data['title'].str.replace(':','_你好_')

end = time.clock()
print(data['title'].str.replace(':','_你好_').head())
print("*"*20)
print('用时{}秒'.format(end-start))

0     EMS_你好_ BACK PAINS/INJURY
1    EMS_你好_ DIABETIC EMERGENCY
2        Fire_你好_ GAS-ODOR/LEAK
3     EMS_你好_ CARDIAC EMERGENCY
4             EMS_你好_ DIZZINESS
Name: title, dtype: object
********************
用时0.2975615410319327秒


In [11]:
# 方法二
start = time.clock()

data['title'].map(lambda x:x.replace(':','_欧啦_'))

end = time.clock()
print(data['title'].map(lambda x:x.replace(':','_欧啦_')).head())
print("|"*20)
print('用时%.20f秒'%(end-start))

0     EMS_欧啦_ BACK PAINS/INJURY
1    EMS_欧啦_ DIABETIC EMERGENCY
2        Fire_欧啦_ GAS-ODOR/LEAK
3     EMS_欧啦_ CARDIAC EMERGENCY
4             EMS_欧啦_ DIZZINESS
Name: title, dtype: object
||||||||||||||||||||
用时0.23902983243235242927秒


# 4. **将分类字符串换为离散变量**

    1. 方法一
        - pd.get_dummies(df['col'])
    2. 方法二
        - 遍历
    3. 方法三
        .map(字典)法
    4. 方法四
        - 机器学习
        - from sklearn.preprocessing import LabelEncoder
        - 返回的是 list
> .map(字典)，可适用于 中英文互转（前提是字典对应）
     

* 大，中，小 转换为  3，2，1

* EMS-1
* Fire-2
* Traffic-3

## 方法一 pd.get_dummies(df['col'])

In [12]:
data['type'].head()

0     EMS
1     EMS
2    Fire
3     EMS
4     EMS
Name: type, dtype: object

In [13]:
pd.get_dummies(data['type']).head()

,EMS,Fire,Traffic
0,1,0,0
1,1,0,0
2,0,1,0
3,1,0,0
4,1,0,0


In [14]:
data.join(pd.get_dummies(data['type'])).tail(2) # 按index关联

,lat,lng,desc,zip,title,timeStamp,twp,addr,e,type,EMS,Fire,Traffic
423907,40.190946,-75.237285,SUMNEYTOWN PIKE & WELLINGTON DR; LOWER GWYNEDD...,19002.0,Traffic: VEHICLE ACCIDENT -,2018-11-16 08:51:48,LOWER GWYNEDD,SUMNEYTOWN PIKE & WELLINGTON DR,1,Traffic,0,0,1
423908,40.155164,-75.264665,HOFFMAN ; LOWER GWYNEDD; 2018-11-16 @ 08:46:25;,19422.0,Traffic: DISABLED VEHICLE -,2018-11-16 08:46:25,LOWER GWYNEDD,HOFFMAN,1,Traffic,0,0,1


## 方法二 遍历

In [15]:
temp_list = data['title'].str.split(": ").tolist()
cate_list = list(set([i[0] for i in temp_list]))
print(cate_list)
print('*'*20)
zeros_df = pd.DataFrame(np.zeros((data.shape[0],len(cate_list))),columns=cate_list)# 构建全零DataFrame
print(zeros_df.head())

# 遍历赋值
for cate in cate_list:
    zeros_df[cate][data['title'].str.contains(cate)] = 1 # 列、行赋值
    
zeros_df.head()

['Fire', 'EMS', 'Traffic']
********************
   Fire  EMS  Traffic
0   0.0  0.0      0.0
1   0.0  0.0      0.0
2   0.0  0.0      0.0
3   0.0  0.0      0.0
4   0.0  0.0      0.0


,Fire,EMS,Traffic
0,0.0,1.0,0.0
1,0.0,1.0,0.0
2,1.0,0.0,0.0
3,0.0,1.0,0.0
4,0.0,1.0,0.0


## 方法三 .map(字典)

In [16]:
name = [1,2,3]
names = (list(set(data['type'])))
dict_name = dict(zip(names,name))
print(dict_name)
data.assign(离散 = data['type'].map(dict_name)).head()

{'Fire': 1, 'EMS': 2, 'Traffic': 3}


,lat,lng,desc,zip,title,timeStamp,twp,addr,e,type,离散
0,40.297876,-75.581294,REINDEER CT & DEAD END; NEW HANOVER; Station ...,19525.0,EMS: BACK PAINS/INJURY,2015-12-10 17:10:52,NEW HANOVER,REINDEER CT & DEAD END,1,EMS,2
1,40.258061,-75.264680,BRIAR PATH & WHITEMARSH LN; HATFIELD TOWNSHIP...,19446.0,EMS: DIABETIC EMERGENCY,2015-12-10 17:29:21,HATFIELD TOWNSHIP,BRIAR PATH & WHITEMARSH LN,1,EMS,2
2,40.121182,-75.351975,HAWS AVE; NORRISTOWN; 2015-12-10 @ 14:39:21-St...,19401.0,Fire: GAS-ODOR/LEAK,2015-12-10 14:39:21,NORRISTOWN,HAWS AVE,1,Fire,1
3,40.116153,-75.343513,AIRY ST & SWEDE ST; NORRISTOWN; Station 308A;...,19401.0,EMS: CARDIAC EMERGENCY,2015-12-10 16:47:36,NORRISTOWN,AIRY ST & SWEDE ST,1,EMS,2
4,40.251492,-75.603350,CHERRYWOOD CT & DEAD END; LOWER POTTSGROVE; S...,NaN,EMS: DIZZINESS,2015-12-10 16:56:52,LOWER POTTSGROVE,CHERRYWOOD CT & DEAD END,1,EMS,2


## 方法四  机器学习

In [17]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(data['type']) # 遍历一遍‘type’列
y = le.transform(data['type']) # 分类字符串转换为离散变量
print(y)

[0 0 1 ... 2 2 2]


# 5. 字符串去重

In [18]:
print(data['type'].unique())
print('*'*20)
print(set(data['type']))
print('*'*20)
print(data['type'].value_counts())

['EMS' 'Fire' 'Traffic']
********************
{'Fire', 'EMS', 'Traffic'}
********************
EMS        208676
Traffic    151458
Fire        63775
Name: type, dtype: int64


# 知识点
1. 数据读取
2. 拆分字符串

  ```python
  Series.str.split(pat=None,
                   n=-1,
                   expand=False)
  ```

  - pat：分隔符，默认为所有空字符，包括空格、换行符\n、制表符\t
  - n：切几刀，默认-1，有多少切多少
  - expand=
    - False，返回Series
    - True，返回DataFrame
    - 结合df1.combine_first(df_new)
      - 注意column的名称

3. 诊断子字符串
    ```python
  df['col'].str.contains('子字符')
  (~df.col1.str.contains('元/平米')).sum()
  ```
4. 替换
    * 
```python
Series.map(lambda x:x.replace("替换谁","替换为"))
```
    * 当然 也可以对 DataFrame进行`df.applymap(lambda x:x.replace("替换谁","替换为"))`
    
    *     
```python
Series.str.replace("替换谁","替换为")
```
5. 分类数据离散化


**数据的种类**

    1. 分类数据（一般通过问卷调查获取）
       - 无序数据
       - 有序数据
    2. 数值数据（一般通过日志获取）
       - 离散数据
       - 连续数据
       
    * 将分类数据比如 小学、初中、高中、大学 分别对应为离散数据 1，2，3，4 
    * 类似MySQL的
```msyql
    case  col
    when '小学' then 1
    when '初中' then 2
    when '高中' then 3
    when '大学' then 4
    else 5
    end  as New_col
```
    * 类似Excel的Vlookup函数
        * 注意，Vlookup的模糊查找和精确匹配的使用条件

    * 方法一，`pd.get_dummies(data['col']).head()`
    * 方法二，遍历赋值
    * 方法三，先构建字典a = {'小学':1,'初中':2,'高中':3,'大学':4}
```python
# 构建字典
dict_name = dict(zip(key值list,values的list))
Series.map(dict_name)
```
        
    * 方法四：机器学习
        * 对分类数据进行 Logistic回归 中使用
        * KNN（K近邻）分类中使用
```python
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(df['col5']) #遍历一边'col5的属性
X = df[['col1','col2','col3']]
y = le.transform(df['col5']) # 分类字符串转换为离散变量

# 交叉检验
from sklearn.model._selection import cross_val_score
scores = cross_val_score(knn,X,y,cv=5,scoing='accuracy')
print("模型转却率为%.2f"%np.mean(scores))
```

> By 儒冠多误身 2019/04/21整理


> 未完待续……

* 正则表达式
* 自然语言处理
* 词云图